<a href="https://colab.research.google.com/github/yigalk89/hatespeech-adversarial-attacks/blob/master/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning

This the following code was used to extract the perturbated sentence from the response of **mixtral-8x7b-32768** model.



In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_json('/content/drive/MyDrive/NLP/test_transformed_data.json')

Mounted at /content/drive


By analyzing the responses of the model, some responses contain not only the transformed sentence but also some redundant comments and notes because of the harmful text.

Therefore, in order to extract the transformed sentence, we use regular expression to search in the response sentence.

However, this automatic method could not clean all the noise in the output. A manual cleaning was conducted after the automated cleaning.

In [2]:
import re
clean = []
original = []
label = []
index = []
for k in range(len(df['transformed_text'])):
  s = df['transformed_text'][k].split('\n\n')
  if len(s) >1:
    for i in range(len(s)):
      #regular expression patterns that contain the transformed data.
      x = re.findall('^Transformed:', s[i])
      y = re.findall('^Here is a',s[i])
      z = re.findall("^Here's a",s[i])
      a = re.findall("^Note",s[i])
      b = re.findall(".Here is the",s[i])
      c = re.findall("^Transformation:",s[i])
      d = re.findall(".T?transformation", s[i])
      m = re.findall('semantic-preserving', s[i])
      n = re.findall('Explanation',s[i])
      o = re.findall('transformation',s[i])
      note = re.findall('Note',s[i])

      # For each case we are checking the all the patterns that apply and the transformed sentence is not duplicated in the sentence
      if m:
        if i+1 <len(s):
          if not re.findall('Explanation',s[i]):
            if s[i+1] not in clean:
              index.append(k)
              original.append(df['original_text'][k])
              clean.append(s[i+1])
              label.append(df['final_label'][k])


      if n:
        if s[i-1] not in clean:
          clean.append(s[i-1])
          index.append(k)
          original.append(df['original_text'][k])
          label.append(df['final_label'][k])

      if o:
        if i+1 <len(s):
          if s[i-1] not in clean:
            clean.append(s[i+1])
            index.append(k)
            original.append(df['original_text'][k])
            label.append(df['final_label'][k])


      if x:
        s1 = re.sub(r"^Transformed:*\s", '',s[i])
        if s1 not in clean:
          original.append(df['original_text'][k])
          index.append(k)
          clean.append(s1)
          label.append(df['final_label'][k])
      #y = re.findall('^Here is a',s[i])
      if y:
        if s[i+1] not in clean:
          original.append(df['original_text'][k])
          index.append(k)
          clean.append(s[i+1])
          label.append(df['final_label'][k])
      #z = re.findall("^Here's a",s[i])
      if z:
        if s[i+1] not in clean:
          original.append(df['original_text'][k])
          index.append(k)
          clean.append(s[i+1])
          label.append(df['final_label'][k])
      #a = re.findall("^Note",s[i])
      if a:
        if s[i-1] not in clean:
          original.append(df['original_text'][k])
          index.append(k)
          clean.append(s[i-1])
          label.append(df['final_label'][k])
      if b:
        if s[i+1] not in clean:
          original.append(df['original_text'][k])
          index.append(k)
          clean.append(s[i+1])
          label.append(df['final_label'][k])
      if c:
        s3 = re.sub(r"^Transformation:*\s", '',s[i])
        if s3 not in clean:
          original.append(df['original_text'][k])
          index.append(k)
          clean.append(s3)
          label.append(df['final_label'][k])
      if d:
        f = re.findall(".note", s[i-1])
        if f:
          if s[i+1] not in clean:
            original.append(df['original_text'][k])
            index.append(k)
            clean.append(s[i+1])
            label.append(df['final_label'][k])
        else:
          if s[i-1] not in clean:
            s_sub = s[i-1].split('\n')
            for j in range(len(s_sub)):
              g = re.findall('^Transformed',s_sub[j])
              if g:
                if s_sub[j] not in clean:
                  original.append(df['original_text'][k])
                  index.append(k)
                  clean.append(s_sub[j])
                  label.append(df['final_label'][k])
              h = re.findall("^Transformation", s_sub[j])
              if h:
                if s_sub[j] not in clean:
                  original.append(df['original_text'][k])
                  index.append(k)
                  clean.append(s_sub[j])
                  label.append(df['final_label'][k])
      if note:
        if s[i-1] not in clean:
          original.append(df['original_text'][k])
          index.append(k)
          clean.append(s[i-1])
          label.append(df['final_label'][k])
  #Other cases there the response contain only one sentence with the transformed one.
  else:
    s2 =  re.sub(r"^Transformed:*\s", '',s[0])
    if s2 not in clean:
      original.append(df['original_text'][k])
      index.append(k)
      clean.append(s2)
      label.append(df['final_label'][k])

In [3]:
len(index) == len(clean)

True

In [4]:
new_df = pd.DataFrame({'original':original, 'transformed_text':clean, 'final_label':label})

In [ ]:
new_df.to_csv('/content/drive/MyDrive/NLP/transformed_clean.csv')